In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
#from matplotlib import rcParam
import seaborn as sb
import os
import sys
sys.setrecursionlimit(1000000)
from scipy.stats import spearmanr
import pathlib
import scipy as sp
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
mcc = MouseConnectivityCache()
from scipy.stats import pearsonr
from scipy.stats import binomtest
import itertools

In [ ]:
from matplotlib import rcParams
#for_binary_barcode_path = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq/Final_processed_sequences/filtered_barcode_matrix_for_binary_junk_test.pkl')
#normalised_barcode_path_junk_test = pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq/Final_processed_sequences/barcode_matrix_normalised_junk_test.pkl')
normalised_barcode_path= pathlib.Path('/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/A1_MAPseq/FIAA32.6a/Sequencing/Processed_data/UpdatedApril/preprocessed_seq_corrected/Final_processed_sequences/barcode_matrix_normalised.pkl')
lcm_reg_dir = pathlib.PurePath('/nemo/lab/znamenskiyp/home/shared/code/MAPseq_processing/AC_MAPseq/Brain1_FIAA32.6a/LCM_registration')
areas = pd.read_csv(lcm_reg_dir/'3d_areas.csv')
RTtosample = pd.read_csv(lcm_reg_dir/'RTprimer_tosample.csv')
areas = areas.merge(RTtosample, how='inner', on='sample')
areas.sort_values("RT_primer", inplace=True)





In [ ]:
#load barcodes, and drop samples that have been qc'd out
barcodes_across_sample = pd.read_pickle(normalised_barcode_path)

areas =areas[areas['RT_primer'].isin(barcodes_across_sample.columns)]

In [ ]:
all_areas_fractions = pd.DataFrame(index=barcodes_across_sample.columns)
all_areas_fractions['name'] = 'string'
areas_to_look = areas.drop('sample', axis=1).set_index('RT_primer')
for i, row in areas_to_look.iterrows():
    row /= row.sum(0)
    
    if row[row.idxmax()]>0.9999:
        name= str(row.idxmax()) + ' ' + str(row[row.idxmax()].round(2))
    else:
        second_highest_row = row.drop(row.idxmax())
        name= str(row.idxmax()) + ' ' + str(row[row.idxmax()].round(2)) + ' ' + str(second_highest_row.idxmax()) + ' ' + str(row[second_highest_row.idxmax()].round(2))
    all_areas_fractions.loc[i] = name
    

Looking at all samples

In [ ]:
columns = barcodes_across_sample.columns.tolist()
#conditional_prob = 
# pd.DataFrame(columns=['A', 'B', 'conditional_probability'])
i = 0
for col_a, col_b in itertools.combinations(columns, 2):
    prob_df = pd.DataFrame()
    prob_df['a'] = barcodes_across_sample[col_a].astype(bool)
    prob_df['b'] = barcodes_across_sample[col_b].astype(bool)
    prob_df['matching'] = prob_df.sum(axis=1)
    probability_a = prob_df['a'].sum()/len(barcodes_across_sample)
    probability_b = prob_df['b'].sum()/len(barcodes_across_sample)
    prob_a_b_null = probability_a*probability_b
    actual = (len(prob_df[prob_df['matching']==2]))/len(barcodes_across_sample)
    p_result = binomtest(len(prob_df[prob_df['matching']==3]), n=len(barcodes_across_sample), p=prob_a_b_null, alternative='two-sided').pvalue
    temp = pd.DataFrame({'A': col_a, 'B': col_b, 'null_probability': prob_a_b_null, 'actual': actual, 'p_val': p_result}, index=[0])
    if i == 0:
        conditional_prob = temp
    else:
        conditional_prob =pd.concat([conditional_prob, temp])
    i = i+1

num_comb = len(conditional_prob)
bonferroni = lambda x: x*num_comb
conditional_prob['p_adj'] = conditional_prob['p_val'].apply(bonferroni)
condional_prob_2_comb = conditional_prob.copy()

In [ ]:
columns = barcodes_across_sample.columns.tolist()
#conditional_prob = 
# pd.DataFrame(columns=['A', 'B', 'conditional_probability'])
i = 0
for col_a, col_b, col_c in itertools.combinations(columns, 3):
    prob_df = pd.DataFrame()
    prob_df['a'] = barcodes_across_sample[col_a].astype(bool)
    prob_df['b'] = barcodes_across_sample[col_b].astype(bool)
    prob_df['c'] = barcodes_across_sample[col_c].astype(bool)
    prob_df['matching'] = prob_df.sum(axis=1)
    probability_a = prob_df['a'].sum()/len(barcodes_across_sample)
    probability_b = prob_df['b'].sum()/len(barcodes_across_sample)
    probability_c = prob_df['c'].sum()/len(barcodes_across_sample)
    prob_a_b_null = probability_a*probability_b*probability_c
    actual = (len(prob_df[prob_df['matching']==3]))/len(barcodes_across_sample)
    p_result = binomtest(len(prob_df[prob_df['matching']==3]), n=len(barcodes_across_sample), p=prob_a_b_null, alternative='two-sided').pvalue
    temp = pd.DataFrame({'A': col_a, 'B': col_b, 'C': col_c, 'null_probability': prob_a_b_null, 'actual': actual, 'p_val': p_result}, index=[0])
    if i == 0:
        conditional_prob = temp
    else:
        conditional_prob =pd.concat([conditional_prob, temp])
    i = i+1
num_comb = len(conditional_prob)
bonferroni = lambda x: x*num_comb
conditional_prob['p_adj'] = conditional_prob['p_val'].apply(bonferroni)
condional_prob_3_comb = conditional_prob.copy()

Looking at only cortical areas

In [ ]:
cortical_areas =[6, 7, 10, 11, 12, 15, 16, 17, 18, 21, 22, 23, 24, 26, 27, 28, 31, 32, 33, 34, 35, 36, 37, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 59, 60, 61, 62, 63, 64, 65, 70, 80, 81, 82, 83, 84]
cortical_areas_adj = list(filter(lambda x: x in barcodes_across_sample.columns, cortical_areas))
#columns = cortical_areas_adj.columns.tolist()
#conditional_prob = 
# pd.DataFrame(columns=['A', 'B', 'conditional_probability'])
i = 0
for col_a, col_b, col_c in itertools.combinations(cortical_areas_adj, 3):
    prob_df = pd.DataFrame()
    prob_df['a'] = barcodes_across_sample[col_a].astype(bool)
    prob_df['b'] = barcodes_across_sample[col_b].astype(bool)
    prob_df['c'] = barcodes_across_sample[col_c].astype(bool)
    prob_df['matching'] = prob_df.sum(axis=1)
    probability_a = prob_df['a'].sum()/len(barcodes_across_sample)
    probability_b = prob_df['b'].sum()/len(barcodes_across_sample)
    probability_c = prob_df['c'].sum()/len(barcodes_across_sample)
    prob_a_b_null = probability_a*probability_b*probability_c
    actual = (len(prob_df[prob_df['matching']==3]))/len(barcodes_across_sample)
    p_result = binomtest(len(prob_df[prob_df['matching']==3]), n=len(barcodes_across_sample), p=prob_a_b_null, alternative='two-sided').pvalue
    total_num = len(prob_df[prob_df['matching']==3])
    predicted_num = prob_a_b_null*len(barcodes_across_sample)
    names_compared = all_areas_fractions.loc[col_a]['name'] + ' & ' + all_areas_fractions.loc[col_b]['name'] + ' & ' + all_areas_fractions.loc[col_c]['name'] 
    temp = pd.DataFrame({'A': col_a, 'B': col_b, 'C': col_c, 'null_probability': prob_a_b_null, 'actual': actual, 'p_val': p_result, 'which_compared': names_compared, 'predicted_num': predicted_num, 'actual_num': total_num}, index=[0])
    if i == 0:
        conditional_prob = temp
    else:
        conditional_prob =pd.concat([conditional_prob, temp])
    i = i+1
num_comb = len(conditional_prob)
bonferroni = lambda x: x*num_comb
conditional_prob['p_adj'] = conditional_prob['p_val'].apply(bonferroni)
condional_prob_3_comb = conditional_prob.copy()
condional_prob_3_comb['effect_size'] = np.log2(condional_prob_3_comb['actual']/condional_prob_3_comb['null_probability'])
condional_prob_3_comb['log_p_adj'] = -np.log10(condional_prob_3_comb['p_adj'])

In [ ]:
condional_prob_3_comb

In [ ]:
over_rep_1 = condional_prob_3_comb[(condional_prob_3_comb['effect_size']>6)&(condional_prob_3_comb['log_p_adj']>150)]
over_rep_2 = condional_prob_3_comb[(condional_prob_3_comb['effect_size']>7)&(condional_prob_3_comb['log_p_adj']>60)]
over_rep_3 = condional_prob_3_comb[(condional_prob_3_comb['effect_size']>8)&(condional_prob_3_comb['log_p_adj']>30)]
over_rep_4 = condional_prob_3_comb[(condional_prob_3_comb['effect_size']>8.5)&(condional_prob_3_comb['log_p_adj']>10)]
under_rep_1 = condional_prob_3_comb[(condional_prob_3_comb['effect_size']<-2)&(condional_prob_3_comb['log_p_adj']>100)]
under_rep_2 = condional_prob_3_comb[(condional_prob_3_comb['effect_size']<-4)&(condional_prob_3_comb['log_p_adj']>20)]
under_rep_3 = condional_prob_3_comb[(condional_prob_3_comb['effect_size']<-3.5)&(condional_prob_3_comb['log_p_adj']>40)]
to_anno = [over_rep_1, over_rep_2, over_rep_3, over_rep_4, under_rep_1, under_rep_2, under_rep_3]

In [ ]:
under_rep_2

In [ ]:
plt.scatter(x= condional_prob_3_comb['effect_size'], y = condional_prob_3_comb['log_p_adj'], s=20)
plt.title('Over- and under-represented motifs of projections between 3 samples containing cortical areas')
for i in to_anno:
    plt.scatter(x = i.effect_size[0], y= i.log_p_adj[0], s=20)

for i in to_anno:
    plt.annotate(i.which_compared[0], (i.effect_size[0], i.log_p_adj[0]+5))


plt.xlabel('log10 effect size (actual/null probability)')
plt.ylabel('-log2 p value')


In [ ]:
i = 0
for col_a, col_b, col_c, col_d in itertools.combinations(cortical_areas_adj, 4):
    prob_df = pd.DataFrame()
    prob_df['a'] = barcodes_across_sample[col_a].astype(bool)
    prob_df['b'] = barcodes_across_sample[col_b].astype(bool)
    prob_df['c'] = barcodes_across_sample[col_c].astype(bool)
    prob_df['d'] = barcodes_across_sample[col_d].astype(bool)
    prob_df['matching'] = prob_df.sum(axis=1)
    probability_a = prob_df['a'].sum()/len(barcodes_across_sample)
    probability_b = prob_df['b'].sum()/len(barcodes_across_sample)
    probability_c = prob_df['c'].sum()/len(barcodes_across_sample)
    probability_d = prob_df['d'].sum()/len(barcodes_across_sample)
    prob_a_b_null = probability_a*probability_b*probability_c*probability_d
    actual = (len(prob_df[prob_df['matching']==4]))/len(barcodes_across_sample)
    p_result = binomtest(len(prob_df[prob_df['matching']==4]), n=len(barcodes_across_sample), p=prob_a_b_null, alternative='two-sided').pvalue
    total_num = len(prob_df[prob_df['matching']==4])
    predicted_num = prob_a_b_null*len(barcodes_across_sample)
    names_compared = all_areas_fractions.loc[col_a]['name'] + ' & ' + all_areas_fractions.loc[col_b]['name'] + ' & ' + all_areas_fractions.loc[col_c]['name'] + ' & ' + all_areas_fractions.loc[col_d]['name'] 
    temp = pd.DataFrame({'A': col_a, 'B': col_b, 'C': col_c, 'D': col_d, 'null_probability': prob_a_b_null, 'actual': actual, 'p_val': p_result, 'which_compared': names_compared, 'predicted_num': predicted_num, 'actual_num': total_num}, index=[0])
    if i == 0:
        conditional_prob = temp
    else:
        conditional_prob =pd.concat([conditional_prob, temp])
    i = i+1


In [ ]:
num_comb = len(conditional_prob)
bonferroni = lambda x: x*num_comb
conditional_prob['p_adj'] = conditional_prob['p_val'].apply(bonferroni)
condional_prob_4_comb = conditional_prob.copy()
condional_prob_4_comb['effect_size'] = np.log2((condional_prob_4_comb['actual']/condional_prob_4_comb['null_probability']))
condional_prob_4_comb['log_p_adj'] = -np.log10(condional_prob_4_comb['p_adj'])

In [ ]:
-(np.log10(condional_prob_4_comb.p_adj.max()))

In [ ]:
condional_prob_4_comb.log_p_adj.min()

In [ ]:
condional_prob_4_comb['effect_size'] = np.log2((condional_prob_4_comb['actual']/condional_prob_4_comb['null_probability']))

In [ ]:
under_rep_1

In [ ]:
over_rep_1 = condional_prob_4_comb[(condional_prob_4_comb['effect_size']>7)&(condional_prob_4_comb['log_p_adj']>175)]
over_rep_2 = condional_prob_4_comb[(condional_prob_4_comb['effect_size']>10)&(condional_prob_4_comb['log_p_adj']>150)]
over_rep_3 = condional_prob_4_comb[(condional_prob_4_comb['effect_size']>12.5)&(condional_prob_4_comb['log_p_adj']>10)]
over_rep_4 = condional_prob_4_comb[(condional_prob_4_comb['effect_size']>10.75)&(condional_prob_4_comb['log_p_adj']>75)]
under_rep_1 = condional_prob_4_comb[(condional_prob_4_comb['effect_size']<-2.29)&(condional_prob_4_comb['log_p_adj']>10)]
under_rep_2 = condional_prob_4_comb[(condional_prob_4_comb['effect_size']<-3)&(condional_prob_4_comb['log_p_adj']>3)]

to_anno = [over_rep_1, over_rep_2, over_rep_3, over_rep_4, under_rep_1, under_rep_2]

In [ ]:
over_rep = condional_prob_4_comb[(condional_prob_4_comb['effect_size']>12)&(condional_prob_4_comb['log_p_adj']>3)&(condional_prob_4_comb['actual_num']>30)]
plt.scatter(x= condional_prob_4_comb['effect_size'], y = condional_prob_4_comb['log_p_adj'], s=20)

for i in to_anno:
    plt.scatter(x = i.effect_size[0], y= i.log_p_adj[0], s=20)
    plt.annotate(i.which_compared[0], (i.effect_size[0], i.log_p_adj[0]+5))
plt.xlabel('log2 effect size (actual/null probability)')
plt.ylabel('-log10 p value')
plt.title('Over- and under-represented motifs of projections between 4 samples containing cortical areas')